In [62]:
import pyblp 
import numpy as np 
import pandas as pd 
import os 
os.chdir("C:/Users/sarah/Dropbox/Upenn Year 3/kraft_heinz")
import statsmodels.api as sm

def generateBLPStyleIV(df   , market_time , market_region , attr_list, func):
    right_df = df.groupby([market_time , market_region] ).apply(lambda x
                                     : func(x[attr_list])).reset_index()
    merged_df = pd.merge(df , right_df , how='left' ,
        on=[market_time , market_region] , suffixes = ['','_total'])
    Z = np.matrix(merged_df[[l + "_total" for l in attr_list]]) - np.matrix(merged_df[attr_list])
    return Z


#def generateHausmanStyleIV(df   , market_time , market_region , attr_list, func):
df = pd.read_csv("demand1.csv")


In [63]:
Z1 = generateBLPStyleIV(df,'market_ids','date_month',['price'], func=np.mean)
Z2 = generateBLPStyleIV(df,'market_ids','date_month',['price'],
                        func=lambda x: np.sqrt(np.mean(x**2)))
Z = np.concatenate([Z1,Z2], axis = 1)
zf = pd.DataFrame(Z, columns = ["demand_instruments" + str(i) for i in range(0,Z.shape[1])])
bf = pd.concat([df,zf],axis = 1)
bf['const']=1
export_csv = bf.to_csv (r'demandZ.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [31]:
###############################################################################################
#logit_formulation = pyblp.Formulation('prices  + if_cainiao + promise_speed',absorb='C(brand_id)')
X1_formulation = pyblp.Formulation('1 + prices',absorb='C(brand_ids)')
X2_formulation = pyblp.Formulation('0 + prices')
product_formulations = (X1_formulation, X2_formulation)
mc_integration = pyblp.Integration('monte_carlo', size=50, seed=0)
bfgs = pyblp.Optimization('bfgs')
problem = pyblp.Problem(product_formulations, bf, integration=mc_integration)
logit_results = problem.solve( optimization=bfgs, sigma=np.ones((1, 1)))reg1 = sm.OLS(bf['diff'],bf[['const', 'lprice','demand_instruments0','demand_instruments1']], missing='drop')
type(reg1)
results = reg1.fit()
type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     615.5
Date:                Sat, 11 May 2019   Prob (F-statistic):               0.00
Time:                        20:00:05   Log-Likelihood:            -1.3449e+05
No. Observations:               68506   AIC:                         2.690e+05
Df Residuals:                   68502   BIC:                         2.690e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.3896    